# Day 9: Disk Fragmenter

## Import libraries

In [1]:
import copy

## Import data

In [56]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - A single string line that represents a disk map.
# - The disk map uses a dense format to represent the layout of files and free space on the disk.
# - The digits alternate between indicating the *length of a file* and the *length of free space*.
# - E.g. A disk map = '12345' represents: 1-block file; 2-blocks of free space; 3-block file; 4-blocks of free space; 5-block file.
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-9_input.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

print(file_data)
# ====================================================================================================================

3862661670678085717870987066944388237122304387269526457922502763481789248860331275352261208057669164151786821267912912843756969762848733948147993167117987933260444111774981131046423127175391198132981556528886466841904153649781277634134462907446885521455261816827393693365787514157407057815067531283609939702491483692118848754199274930671713112463785536324065963870335246678963763726482884525850924546326162806111582168467928714746722135549182813311357913767784772057532215509849329438692631404536853075567677649217971898253752889665535729441681394169328065824974875236246829434715368379917054666169484640328312687686856934686335482475418948464312408116998373542665883885409969358775724793145487879242845351955713129714201538831838815217875978893510632238164136339832353066972996532394316253993598292538992698265645609135462016793077823110238350549616181835454778658492274752133885133617848131772264841284426255741812575847706824989369827790814863999436844435832394788125212563539120887592164845927156

## Helper functions

In [36]:
def rearrange_disk_map_p1(dm):
    rearrangedDM = copy.deepcopy(dm)  # Create a deep (indep.) copy of the disk map to avoid modifying the original
    left = 0  # Start from the beginning of the string
    right = len(rearrangedDM) - 1  # Start from the end of the string

    while left < right:
        # If the current character is a '.', find the first number from the right
        if rearrangedDM[left] == '.':
            # Traverse from the right to find the first number
            while right > left and rearrangedDM[right] == '.':
                right -= 1
                
            # Swap the '.' with the number
            if right > left:
                rearrangedDM[left], rearrangedDM[right] = rearrangedDM[right], rearrangedDM[left]
                right -= 1

        left += 1 # END_WHILE

    # Return the rearranged string
    return rearrangedDM

In [ ]:
def rearrange_disk_map_p1_alt(dm):
    # Parse the disk map into blocks
    blocks = []
    file_id = 0
    is_file = True  # starts with file
    i = 0
    
    while i < len(dm):
        length = int(dm[i])

        if is_file:
            blocks.extend([str(file_id)] * length)
            file_id += 1
        else:
            blocks.extend(['.'] * length)

        is_file = not is_file
        i += 1
    
    # Simulate compaction
    changed = True

    while changed:
        changed = False
        # Find the first free space
        first_free = None

        for pos in range(len(blocks)):
            if blocks[pos] == '.':
                first_free = pos
                break

        if first_free is None:
            break  # no free space left

        # Find the last file block to move
        last_file_pos = None

        for pos in range(len(blocks) - 1, -1, -1):
            if blocks[pos] != '.':
                last_file_pos = pos
                break

        if last_file_pos is not None and last_file_pos > first_free:
            # Move the block
            blocks[first_free] = blocks[last_file_pos]
            blocks[last_file_pos] = '.'
            changed = True
    
    return blocks

In [59]:
def rearrange_disk_map_p2(dm):
    #print(f"Original: {dm}")

    arrDM = list(dm)
    #print(f"Chars: {arrDM}")
    
    left = 0
    right = len(arrDM) - 1
    iteration = 1
    fileBlocksVisited = []

    # NOTE: This loop should repeat while the leftmost file block number (0) has NOT been reached
    while left < right: 
        #print(f"\n--- Iteration {iteration} ---")
        #print(f"Searching from left (starting at index {left})...")
        
        # Find free space block (consecutive dots) from the left (L) -->
        freeSpaceBlockIdx_Start = left
        while freeSpaceBlockIdx_Start < len(arrDM) and arrDM[freeSpaceBlockIdx_Start] != '.':
            freeSpaceBlockIdx_Start += 1
        
        freeSpaceBlockIdx_End = freeSpaceBlockIdx_Start # intiialize end index to the start index
        while freeSpaceBlockIdx_End < len(arrDM) and arrDM[freeSpaceBlockIdx_End] == '.':
            freeSpaceBlockIdx_End += 1 # REMEMBER: end index is exclusive (while loop goes one index too far at the end)
        
        freeSpaceBlockSize = freeSpaceBlockIdx_End - freeSpaceBlockIdx_Start
        
        if freeSpaceBlockSize == 0:
            #print("No dots found - stopping")
            break
        
        #print(f"Found {freeSpaceBlockSize} dots at positions {freeSpaceBlockIdx_Start}-{freeSpaceBlockIdx_End - 1}")
        
        # Find file block (Chain of Same Numbers = CSN) from the right (R) <--
        #print(f"Searching from right (starting at index {right})...")
        fileBlockIdx_Start = right
        while fileBlockIdx_Start >= 0 and (not (arrDM[fileBlockIdx_Start].isdigit())):
            fileBlockIdx_Start -= 1
        
        if fileBlockIdx_Start < 0:
            #print("No numbers found - stopping")
            break
            
        # Find the start index of the file block
        fileBlockIdx_End = fileBlockIdx_Start # initialize end index to the start index
        fileBlockNum = arrDM[fileBlockIdx_Start]

        # NOTE: Since we are traversing from the right, move left while the character is the same number
        # - Therefore, the end of the CSN from the right = the file block start index
        while fileBlockIdx_Start >= 0 and arrDM[fileBlockIdx_Start] == fileBlockNum:
            fileBlockIdx_Start -= 1
        
        fileBlockIdx_Start += 1 # adjust back to the first occurrence of the file block number since the while loop condition above will have moved one index too far left
        fileBlockSize = fileBlockIdx_End - fileBlockIdx_Start + 1
        
        #print(f"Found {fileBlockSize} '{fileBlockNum}'s at positions {fileBlockIdx_Start}-{fileBlockIdx_End}")

        fileBlock = arrDM[fileBlockIdx_Start:(fileBlockIdx_End + 1)]
        #print(f"File block to consider moving: {''.join(fileBlock)}")   

        # NOTE: Since we should only move file blocks exactly once, we should not attempt to move a file block multiple times
        # - Therefore, already visited file blocks should not be considered again
        if fileBlock in fileBlocksVisited:
            #print(f"File block '{fileBlock}' already visited - skipping")
            right = fileBlockIdx_Start - 1 # Move the right pointer leftwards to continue searching for the next file block
        # NOTE: Edge case - do not attempt to move the file block if it is already at index 0 (leftmost position of disk map)
        elif fileBlockIdx_Start != 0:
            # NOTE: Check if the free space block (...) and the file block (CSN) can be swapped:
            # - COND-1: free space block size > file block size
            # - COND-2: free space block is located BEFORE the file block
            if freeSpaceBlockSize >= fileBlockSize and (freeSpaceBlockIdx_End < fileBlockIdx_Start): 
                #print(f"Swapping: {freeSpaceBlockSize} dots > {fileBlockSize} numbers")

                # NOTE: Add the current file block being swapped to the visited list since we do not want to swap it again
                fileBlocksVisited.append(fileBlock)
                #print("Added to visited list:", fileBlocksVisited)  

                # NOTE: Swap the dots (free space block) with numbers (file block)
                # - Replace dots from the first index of dots to the length of numbers in the found file block (E.g. '999' = 3 chars)
                # - This means that we only replace as many dots as there are numbers in the file block
                arrDM[freeSpaceBlockIdx_Start:(freeSpaceBlockIdx_Start + len(fileBlock))] = fileBlock
                #print(arrDM[fileBlockIdx_Start:(fileBlockIdx_End + 1)])

                # Swap the numbers (file block) with dots (free space block)
                arrDM[fileBlockIdx_Start:(fileBlockIdx_End + 1)] = '.' * len(fileBlock) # acts like 'replace()'
                #print(arrDM[fileBlockIdx_Start:(fileBlockIdx_End + 1)])
                
                #print(f"After swap: {''.join(arrDM)}")
                
                # RESET L & R pointers
                left = 0 # Move left pointer to start idx of disk map
                right = fileBlockIdx_Start - 1 # Move the right pointer leftwards to continue searching for the *next file block
            else:
                #print(f"No swap: {freeSpaceBlockSize} dots <= {fileBlockSize} numbers")
                left = freeSpaceBlockIdx_End # Move left pointer rightwards past the current free space block
        else:
            #print("File block is already at leftmost position - skipping")
            right = fileBlockIdx_Start - 1 # Move the right pointer leftwards to continue searching for the next file block    

        # NOTE: IFF the left pointer has crossed the right pointer, then we need to reset the pointers and add the current file block to the visited list:
        # - Since this means that all free space blocks from the left side have been traversed for this file block on the right and NO swaps were possible
        # - Therefore we need to move on to search for a swap for the NEXT file block and if the *same file block is encountered again, it should be skipped since it has already been searched
        if left > right and arrDM[right] != '0': 
            fileBlocksVisited.append(fileBlock)
            #print("No space found - added to visited list:", fileBlocksVisited)  
            left = 0 # Move left pointer to start idx of disk map
            right = fileBlockIdx_Start - 1 # Move the right pointer leftwards to continue searching for the *next file block
        
        iteration += 1
        
    rearrangedDM = ''.join(arrDM)
    #print(f"\nRe-arranged DM: {rearrangedDM}")
    
    return rearrangedDM
# -------------------------------------------------------------------------------------
# *** TEST CASES: ***
# rearrange_disk_map_p2('1....33..44...555.666')
# rearrange_disk_map_p2('00...111...2...333.44.5555.6666.777.888899')

## Part 1

In [57]:
# *** [PART 1] ***
# ! PROBLEM: The amphipod would like to move file blocks *one at a time* from the END of the disk to the LEFTMOST free space block (until there are no gaps remaining between file blocks - see examples on website).
# - Each file on disk also has an ID number based on the *order* of the files as they appear BEFORE they are rearranged, starting with ID: 0.
# - E.g. A disk map '12345' has 3 files: 1-block file (ID: 0); 3-block file (ID: 1); 5-block file (ID: 2).
# - - 'x' ID digits are used to represent EACH block (where 'x' = file block number) & '.' = free space.
# - E.g. A disk map '12345' becomes '0..111....22222' after rearrangement.
# - TODO: Calculate the resulting filesystem checksum of the RE-ARRANGED & MOVED disk map: Add up the result of multiplying each of the blocks' position with the file ID number it contains.
# - E.g. '0099811188827773336446555566...' = '(0 * 0 = 0) + (1 * 0 = 0) + (2 * 9 = 18) + (3 * 9 = 27) + (4 * 8 = 32) + ... ' (Multiply array item idx by value)
# ====================================================================================================================
# ! Create a deep (independent) copy of the data, such that changes made to the copy do not affect the original data to still test/re-run Part 1/2 with the correct INITIAL (and not modified) data
# - NOTE: Not using a deep copy will modify the original data after running Part 1/2, therefore no correct output will be calculated in repeated runs.
disk_map = copy.deepcopy(file_data)
arrTransformedDiskMap = []
rearrangedDiskMap = ''
blockCounter = 0
checkSum = 0

for i in range(len(disk_map)):
    if i % 2 == 0: # If the index is even (i.e., we're looking at a block file)
        for j in range(int(disk_map[i])): #'x' ID digits are used to represent EACH block (where 'x' = file block number)
            # NOTE: DO NOT use 'transformedDiskMap +=' because if 'blockCounter' number > 1 digit, then when transform string to a list, it will break up all numbers in the string into singluar digits (E.g. '10' => '1','0'), therefore append each number AS A WHOLE into an array list
            arrTransformedDiskMap.append(str(blockCounter))
            
        blockCounter += 1
    elif i % 2 != 0: # If the index is odd (i.e., we're looking at a free block space)
        for j in range(int(disk_map[i])):
            arrTransformedDiskMap.append('.') # Append '.' as many times as the CURRENT free block space number

""" Re-arrange transformed disk map """
# Move file blocks (ID numbers) from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks)
rearrangedDiskMap = rearrange_disk_map_p1(arrTransformedDiskMap)
# rearrangedDiskMap_alt = rearrange_disk_map_p1_alt(disk_map)
print("Original disk map:", '\t', ''.join(arrTransformedDiskMap))
print("Re-arranged disk map:", '\t',''.join(rearrangedDiskMap))
# print(rearrangedDiskMap_alt)

for i in range(len(rearrangedDiskMap)):
    if rearrangedDiskMap[i] != '.':
        checkSum += (i * int(rearrangedDiskMap[i]))

print("Filesystem checksum (Part 1):", checkSum)
# ====================================================================================================================

Original disk map: 	 000........111111..222222......3......4444444555555.......6666666677777777.....8888888.9999999........10101010101010111111111111111111........12121212121212131313131313......141414141414141414....15151515...1616161616161616........1717...18181818181818.1919..20202021212121...2222222222222222.......2323......242424242424242424.....2525......26262626.....27272727272727.........2828..29292929293030.......313131313131...32323232........33.......3434343434343434.........3535....3636363636363636........373737373737383838...39..40404040404040.....414141.....4242..434343434343.444445454545454545454646464646.......474747474747......484848484848484848.494949494949....50.....51.......5252525252525252......5353535353535353..54..555555555555.......565656565656565656.5757.........58..5959595959595959....606060.......6161616161......626262626262626262......636363636363636363.......646464646464..6565656565656565....6666666666666666.......676767...686868686868686868....696969696969

## Part 2

In [60]:
# *** [PART 2] ***
# ! PROBLEM: Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?
# - TODO: Rather than moving individual blocks, compact disk files by moving WHOLE files instead.
# - Attempt to move *whole files* to the *leftmost span* of free space blocks that could fit the file.
# - Attempt to move each file *exactly once* in order of *decreasing file ID number*, starting with the file with the *highest file ID number*.
# - If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.
#====================================================================================================================
# ! Create a deep (independent) copy of the data, such that changes made to the copy do not affect the original data to still test/re-run Part 1/2 with the correct INITIAL (and not modified) data
# - NOTE: Not using a deep copy will modify the original data after running Part 1/2, therefore no correct output will be calculated in repeated runs.
disk_map = copy.deepcopy(file_data)
arrTransformedDiskMap_p2 = []
arrBlockIdx = []
rearrangedDiskMap_p2 = ''
blockCounter_p2 = 0
checkSum_p2 = 0

for i in range(len(disk_map)):
    if i % 2 == 0: # If the index is even (i.e., we're looking at a block)
        j = int(disk_map[i]) #'x' ID digits are used to represent EACH block (where 'x' = file block number)
        # NOTE: DO NOT use 'transformedDiskMap +=' because if 'blockCounter' number > 1 digit, then when transform string to a list, it will break up all numbers in the string into singluar digits (E.g. '10' => '1','0'), therefore append each number AS A WHOLE into an array list
        arrTransformedDiskMap_p2.extend(str(blockCounter_p2) * j) # Append the CURRENT blockCounter_p2 number AS MANY times as the CURRENT block size 'j' using 'extend()'

        blockCounter_p2 += 1
        arrBlockIdx.append(blockCounter_p2 - 1) # Track the file block IDs
    elif i % 2 != 0: # If the index is odd (i.e., we're looking at a free space)
        j = int(disk_map[i])
        arrTransformedDiskMap_p2.extend('.' * j) # Append '.' as many times as the CURRENT free space number 'j' using 'extend()'

""" Re-arrange transformed disk map """
# Move file blocks (Groups of same ID numbers) ONCE from the end of the disk to the leftmost free space block
rearrangedDiskMap_p2 = rearrange_disk_map_p2(arrTransformedDiskMap_p2)
print("Original disk map:", '\t',''.join(arrTransformedDiskMap_p2))
print("Re-arranged disk map:", '\t',''.join(rearrangedDiskMap_p2))

for i in range(len(rearrangedDiskMap_p2)):
    if rearrangedDiskMap_p2[i] != '.':
        checkSum_p2 += (i * int(rearrangedDiskMap_p2[i]))

print("Filesystem checksum (Part 2):", checkSum_p2)



Original disk map: 	 000........111111..222222......3......4444444555555.......6666666677777777.....8888888.9999999........10101010101010111111111111111111........12121212121212131313131313......141414141414141414....15151515...1616161616161616........1717...18181818181818.1919..20202021212121...2222222222222222.......2323......242424242424242424.....2525......26262626.....27272727272727.........2828..29292929293030.......313131313131...32323232........33.......3434343434343434.........3535....3636363636363636........373737373737383838...39..40404040404040.....414141.....4242..434343434343.444445454545454545454646464646.......474747474747......484848484848484848.494949494949....50.....51.......5252525252525252......5353535353535353..54..555555555555.......565656565656565656.5757.........58..5959595959595959....606060.......6161616161......626262626262626262......636363636363636363.......646464646464..6565656565656565....6666666666666666.......676767...686868686868686868....696969696969